In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("https://covid19.manaus.am.gov.br/wp-content/uploads/Manaus.csv", sep=';', 
                    parse_dates=['_dt_evolucao', '_dt_notificacao', '_dt_sintomas']
                   )

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16,17,18,22,23,26,27,28,29,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,_dt_evolucao,_dt_notificacao,_dt_sintomas,_classificacao,_conclusao,_origem,_comorb_cardio,_comorb_cromossomica,_comorb_diabetes,_comorb_hemato,_comorb_hepatica,_comorb_imuno,_comorb_neurologica,_comorb_obessidade,_comorb_renal,_comorb_respiratoria,_criterio,_etnia,_evolução,_idade,_faixa etária,_gestante,_profiss_saude,_raca,_se_notificacao,_sexo,_sintoma_dispneia,_sintoma_febre,_sintoma_garganta,_sintoma_tosse,_sintoma_outros,_srag,_teste_anticorpo,_teste_antigeno,_teste_pcr,_bairro,_bairro_mapa,_distrito,_taxa
0,2020-01-25,2020-03-01,2020-01-01,Descartado,Recuperado,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado,4.0,1a - 4a,NaN,NaN,Parda,1.0,M,Sim,Sim,Sim,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,TARUMA,TARUMÃ,OESTE,"0,243042897"
1,2020-03-01,2020-03-01,2020-01-01,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado,26.0,20a - 29a,NaN,NaN,Parda,1.0,F,Sim,Sim,Sim,Sim,Sim,Influenza B,NaN,NaN,NaN,TARUMA-ACU,TARUMÃ-AÇÚ,OESTE,"0,684931507"
2,2020-05-01,2020-03-01,2020-03-01,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado em UTI,27.0,20a - 29a,NaN,NaN,Parda,1.0,M,Sim,Sim,Sim,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,JORGE TEIXEIRA,JORGE TEIXEIRA,LESTE,"0,073137374"
3,2020-01-16,2020-08-01,2020-04-01,Descartado,Óbito,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado em UTI,94.0,80a ou +,NaN,NaN,Parda,2.0,M,Sim,Sim,Sim,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,FLORES,FLORES,SUL,"0,198306463"
4,2020-11-01,2020-08-01,2020-05-01,Descartado,Recuperado,"SIVEP,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laboratorial,NaN,Internado,0.0,< 1a,NaN,NaN,Parda,2.0,M,Sim,Sim,NaN,Sim,Sim,SRAG não especificado,NaN,NaN,NaN,CIDADE DE DEUS,CIDADE DE DEUS,NORTE,"0,117699678"


# Visão Geral dos Casos Confirmados

Filtrando apenas casos confirmados

In [4]:
data['_classificacao'].value_counts()

Descartado    126563
Confirmado     80420
Em análise     37001
Name: _classificacao, dtype: int64

In [5]:
data = data[data['_classificacao'] == 'Confirmado']

In [6]:
data.sample(5)

,_dt_evolucao,_dt_notificacao,_dt_sintomas,_classificacao,_conclusao,_origem,_comorb_cardio,_comorb_cromossomica,_comorb_diabetes,_comorb_hemato,_comorb_hepatica,_comorb_imuno,_comorb_neurologica,_comorb_obessidade,_comorb_renal,_comorb_respiratoria,_criterio,_etnia,_evolução,_idade,_faixa etária,_gestante,_profiss_saude,_raca,_se_notificacao,_sexo,_sintoma_dispneia,_sintoma_febre,_sintoma_garganta,_sintoma_tosse,_sintoma_outros,_srag,_teste_anticorpo,_teste_antigeno,_teste_pcr,_bairro,_bairro_mapa,_distrito,_taxa
102290,NaT,2020-11-26,2020-01-11,Confirmado,NaN,"eSUS VE,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,30a - 39a,NaN,NaN,Parda,48.0,F,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,CIDADE NOVA,CIDADE NOVA,NORTE,"0,068152389"
243737,NaT,2020-11-26,NaT,Confirmado,NaN,GAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.0,30a - 39a,NaN,NaN,NaN,48.0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,JAPIM 2,JAPIIM,SUL,"0,173837462"
112282,2020-10-06,2020-10-06,2020-01-05,Confirmado,Recuperado,"eSUS VE,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.0,50a - 59a,NaN,NaN,Parda,24.0,M,NaN,Sim,NaN,NaN,NaN,NaN,1.0,NaN,NaN,ZUMBI DOS PALMARES,ZUMBI DOS PALMARES,LESTE,"0,234807927"
163756,2020-09-11,2020-10-23,2020-10-20,Confirmado,Recuperado,"eSUS VE,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Em tratamento domiciliar,49.0,40a - 49a,NaN,NaN,Ignorado,43.0,M,NaN,NaN,NaN,Sim,Sim,NaN,NaN,NaN,1.0,PONTA NEGRA,PONTA NEGRA,OESTE,"1,648804617"
225175,2020-07-23,2020-10-12,2020-02-12,Confirmado,NaN,"eSUS VE,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,30a - 39a,NaN,NaN,Branca,50.0,F,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,REDENÇÃO,REDENÇÃO,OESTE,"0,234763828"


**Quantos atributos descrevem cada exemplo? Quais são eles?**

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80420 entries, 130 to 243982
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   _dt_evolucao          47739 non-null  datetime64[ns]
 1   _dt_notificacao       80395 non-null  datetime64[ns]
 2   _dt_sintomas          78922 non-null  datetime64[ns]
 3   _classificacao        80420 non-null  object        
 4   _conclusao            42774 non-null  object        
 5   _origem               80420 non-null  object        
 6   _comorb_cardio        6071 non-null   object        
 7   _comorb_cromossomica  185 non-null    object        
 8   _comorb_diabetes      4763 non-null   object        
 9   _comorb_hemato        100 non-null    object        
 10  _comorb_hepatica      84 non-null     object        
 11  _comorb_imuno         813 non-null    object        
 12  _comorb_neurologica   297 non-null    object        
 13  _comorb_obess

**Quantos casos confirmados há em Manaus, cumulativamente?**

80.420

**A qual período de tempo a base de dados se refere, isto é, qual o registro mais antigo e qual o mais recente? Leve em conta a data de notificação.**

Caso mais antigo: 04/01/2020

In [8]:
data['_dt_notificacao'].min()

Timestamp('2020-01-04 00:00:00')

Caso mais recente: 29/12/2020

In [9]:
data['_dt_notificacao'].max()

Timestamp('2020-12-29 00:00:00')

# Limpeza de dados

Vamos remover atributos que não são úteis para as análises propostas neste projeto, assim como remover linhas em que aparecem campos vazios (`NaN`, `NaT`).

Removendo colunas com informações irrelevantes, o que inclui: `_dt_evolucao`, `_dt_sintomas`, `_profiss_saude`, `_etnia`, `_raca`, `_gestante`, `_se_notificacao`, `_srag`, `_taxa`, `_criterio`, `_origem`, `_evolução` e todas as colunas que começam com `_sintoma` ou `_comorb`.


In [11]:
data['_teste_anticorpo'].fillna(0, inplace = True)
data['_teste_antigeno'].fillna(0, inplace = True)
data['_teste_pcr'].fillna(0, inplace = True)


In [12]:
columns_to_remove = ['_dt_evolucao', '_dt_sintomas', '_profiss_saude', 
                     '_etnia', '_raca', '_gestante', '_se_notificacao', 
                     '_srag', '_taxa', '_criterio', '_origem', '_evolução']

columns_to_remove += [
                        column for column in data.columns 
                        if column.startswith('_sintoma') 
                        or column.startswith('_comorb') 
                      ]

In [13]:
data = data.drop(columns_to_remove, axis=1)

Removendo todas as linhas que possuem valores nulos, como `NaN` e `NaT` (para os casos de colunas com dados temporais).

In [14]:
data = data.dropna()

In [15]:
data.to_csv('Manaus-preprocessed.csv', sep=';')

# Análises pós-limpeza

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41675 entries, 130 to 225161
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   _dt_notificacao   41675 non-null  datetime64[ns]
 1   _classificacao    41675 non-null  object        
 2   _conclusao        41675 non-null  object        
 3   _idade            41675 non-null  float64       
 4   _faixa etária     41675 non-null  object        
 5   _sexo             41675 non-null  object        
 6   _teste_anticorpo  41675 non-null  float64       
 7   _teste_antigeno   41675 non-null  float64       
 8   _teste_pcr        41675 non-null  float64       
 9   _bairro           41675 non-null  object        
 10  _bairro_mapa      41675 non-null  object        
 11  _distrito         41675 non-null  object        
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 4.1+ MB


In [17]:
data = pd.read_csv('Manaus-preprocessed.csv', sep=';')

**Qual a porcentagem de indivíduos recuperados em relação ao todo?**

91,89%

In [18]:

data["_conclusao"].value_counts(normalize=True) * 100

Recuperado                 91.892022
Óbito                       7.973605
Óbito por outras causas     0.134373
Name: _conclusao, dtype: float64

**Os casos acometeram mais indivíduos do sexo masculino ou feminino?**

Feminino

In [19]:
data.groupby(by="_sexo").size()

_sexo
F    22357
M    19318
dtype: int64

**Qual a média e desvio padrão de idade dos indivíduos que contraíram COVID-19? Qual o indivíduo mais jovem e o mais idoso a contraírem tal enfermidade?**

Média: 43,48

Desvio Padrão: 17,38

Indivíduo mais jovem: 0

Indivíduo mais idoso: 120

In [20]:
data['_idade'].mean()

43.47652069586083

In [21]:
data['_idade'].std()

17.376358811650032

In [22]:
data['_idade'].min()

0.0

In [23]:
data['_idade'].max()

120.0

**Qual o bairro com maior incidência de casos?**

São José Operário

In [30]:
data.groupby(by="_bairro_mapa").size().sort_values(ascending=False)

_bairro_mapa
SÃO JOSÉ OPERÁRIO          2351
CIDADE NOVA                2318
ALVORADA                   2015
COMPENSA                   2008
JORGE TEIXEIRA             1987
                           ... 
NOSSA SENHORA APARECIDA      76
RIO NEGRO                    24
BR-174                       24
AM-010                        6
RIO TARUMÃ-MIRIM              3
Length: 67, dtype: int64

**Quais os três bairros com maior incidência de casos recuperados?**



1.   São José Operário
2.   Cidade Nova
3.   Alvorada



In [49]:
data.groupby(["_bairro_mapa", "_conclusao"]).size().sort_values(ascending=False)

_bairro_mapa         _conclusao             
SÃO JOSÉ OPERÁRIO    Recuperado                 2234
CIDADE NOVA          Recuperado                 2079
ALVORADA             Recuperado                 1872
JORGE TEIXEIRA       Recuperado                 1863
COMPENSA             Recuperado                 1846
                                                ... 
FLORES               Óbito por outras causas       1
DA PAZ               Óbito por outras causas       1
CRESPO               Óbito por outras causas       1
COMPENSA             Óbito por outras causas       1
MONTE DAS OLIVEIRAS  Óbito por outras causas       1
Length: 166, dtype: int64

**Quais os tipos de testes efetuados, segundo os dados? Indique os dados de maneira quantitativa e percentual.**

In [50]:
teste_anticorpo = len(data[data["_teste_anticorpo"] > 0])
print("Teste Anticorpo:", teste_anticorpo, "\nPorcentagem:",100*teste_anticorpo/len(data))
teste_antigeno = len(data[data["_teste_antigeno"] > 0])
print("\nTeste Antigeno:", teste_antigeno, "\nPorcentagem:",100*teste_antigeno/len(data))
teste_pcr = len(data[data["_teste_pcr"] > 0])
print("\nTeste PCR:", teste_pcr, "\nPorcentagem:",100*teste_pcr/len(data))

Teste Anticorpo: 25962 
Porcentagem: 62.29634073185363

Teste Antigeno: 4053 
Porcentagem: 9.725254949010198

Teste PCR: 11798 
Porcentagem: 28.309538092381523


**Qual taxa de letalidade pode ser calculada a partir do conjunto de dados? Para calcular esta taxa, considere a fração do total de óbitos pelo total de casos;**

In [52]:
100 * len(data[data["_conclusao"] == "Óbito"])/len(data)

7.973605278944211

**Qual o tipo de correlação, mediante coeficiente de correlação de Pearson, entre a idade e o número de casos? Para responder a esta pergunte, agrupe o número de casos por idade e efetue o cálculo de tal coeficiente. Indique, a partir do resultado, a natureza desta correlação, se é positiva ou negativa, e qual sua intensidade.**

É uma correlação negativa, com intensidade igual a -0,345

In [55]:
idade = data.groupby(by="_idade").size().reset_index(name='qtde')

In [58]:
idade.corr()

,_idade,qtde
_idade,1.000000,-0.345031
qtde,-0.345031,1.000000
